In [65]:
# close chrome driver by calling quit()
driver.quit()

In [58]:
from selenium import webdriver
import time

In [59]:
http_proxy="localhost:8118"

In [60]:
options = webdriver.ChromeOptions()
# MacOS
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
# Linux
# options.binary_location = '/usr/bin/google-chrome-stable'

# comment headless argument for debugging/ visualization
# options.add_argument('headless')
options.add_argument('window-size=1200x600')
# add proxy, restarting chrome needed if not work
options.add_argument('--proxy-server={0}'.format(http_proxy))
driver = webdriver.Chrome(chrome_options=options)

In [61]:
def scroll_down(driver):
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        print("scroll down event")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Click show more button if any
        show_more_btn = driver.find_element_by_css_selector('#smb')
        try:
            print(show_more_btn.get_attribute("outerHTML"))
            show_more_btn.click()
        except:
            x = 0

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [34]:
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
driver.get('https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592&cl=2&lm=-1&st=-1&fm=index&fr=&hs=0&xthttps=111111&sf=1&fmq=&pv=&ic=0&nc=1&z=&se=1&showtab=0&fb=0&width=&height=&face=0&istype=2&ie=utf-8&word=%E9%A3%8E%E6%99%AF')

scroll_down(driver)

scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event
scroll down event


KeyboardInterrupt: 

In [ ]:
# basic operations
driver.get('https://www.google.com/imghp?hl=en&tab=wi&authuser=0')
# chrome extension, get xpath of element, http://selectorgadget.com/
search = driver.find_element_by_css_selector('input#lst-ib')
click = driver.find_element_by_css_selector('#sfdiv :nth-child(1)')
search.send_keys("golang")
click.click()

In [4]:
# save image
driver.get_screenshot_as_file('main-page2.png')

True

![title](main-page2.png)

In [62]:
def download_pagev2(url):
    driver.get(url)
    scroll_down(driver)
    elem = driver.find_element_by_xpath("//*")
    page = elem.get_attribute("outerHTML")
    return page

In [63]:
# '<html><head></head><body></body></html>' 返回空白页面，就google域名是空白？
raw_html = download_pagev2("https://www.google.com/search?q=Brinjal%20Fruit%20Borer&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&tbs=&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg")

scroll down event
<input class="ksb huRCDf" value="显示更多结果" id="smb" data-lt="正在加载..." type="button" data-ved="0ahUKEwisgrnl9IzaAhVKpZQKHVFnAPAQxdoBCFE">
scroll down event
<input class="ksb huRCDf" value="显示更多结果" id="smb" data-lt="正在加载..." type="button" data-ved="0ahUKEwisgrnl9IzaAhVKpZQKHVFnAPAQxdoBCFE">
scroll down event
<input class="ksb huRCDf" value="显示更多结果" id="smb" data-lt="正在加载..." type="button" data-ved="0ahUKEwisgrnl9IzaAhVKpZQKHVFnAPAQxdoBCFE">
scroll down event
<input class="ksb huRCDf" value="显示更多结果" id="smb" data-lt="正在加载..." type="button" data-ved="0ahUKEwisgrnl9IzaAhVKpZQKHVFnAPAQxdoBCFE">


In [25]:
def _images_get_next_item(s):
    start_line = s.find('rg_di')
    if start_line == -1:  # If no links are found then give an error!
        end_quote = 0
        link = "no_links"
        return link, end_quote
    else:
        start_line = s.find('"class="rg_meta"')
        start_content = s.find('"ou"', start_line + 1)
        end_content = s.find(',"ow"', start_content + 1)
        content_raw = str(s[start_content + 6:end_content - 1])
        return content_raw, end_content


# Getting all links with the help of '_images_get_next_image'
def _images_get_all_items(page):
    items = []
    while True:
        item, end_content = _images_get_next_item(page)
        if item == "no_links":
            break
        else:
            items.append(item)  # Append all the links in the list named 'Links'
            time.sleep(0.1)  # Timer could be used to slow down the request for image downloads
            page = page[end_content:]
    return items

In [ ]:
len(_images_get_all_items(raw_html))